## 1. Librerías

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate

## 2. Instanciar un Objeto de Modelo de Lenguaje de OpenAI (De Paga)

In [2]:
llm = AzureChatOpenAI(
    azure_endpoint= "https://openai-mr.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview",
    api_version= '2024-08-01-preview',
    temperature= 0.7,
    max_tokens= 100,
)

## 3. Comma Separated List (Lista Separada por Comma)

In [4]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [5]:
# Crear un objeto de CommaSeparatedListOutputParser.
formato_output = CommaSeparatedListOutputParser()

In [6]:
formato_instrucciones = formato_output.get_format_instructions()

In [9]:
formato_instrucciones

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [15]:
prompt = PromptTemplate(
    template= "Provee 5 ejemplos de {query}.\{formato_instrucciones}",
    input_variables= ["query"],
    partial_variables= {"formato_instrucciones": formato_instrucciones} # partial_variables es un diccionario que contiene las variables que se van a reemplazar en el template.
)

In [16]:
prompt = prompt.format(query= "frutas")

In [18]:
print(prompt)

Provee 5 ejemplos de frutas.\Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [19]:
output = llm.invoke(prompt)

In [21]:
print(output.content)

manzana, plátano, naranja, fresa, uva


## 4. Formato JSON

In [38]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [39]:
response_esquemas = [
    ResponseSchema(name= "moneda", description= "Respuesta a la pregunta del usuario"),
    ResponseSchema(name= "abreviatura", description= "¿Cúal es la abreviatura de la moneda?"),
]

In [40]:
formato_output = StructuredOutputParser.from_response_schemas(response_esquemas)

In [41]:
print(formato_output)

response_schemas=[ResponseSchema(name='moneda', description='Respuesta a la pregunta del usuario', type='string'), ResponseSchema(name='abreviatura', description='¿Cúal es la abreviatura de la moneda?', type='string')]


In [42]:
formato_instrucciones = formato_output.get_format_instructions()

print(formato_instrucciones)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"moneda": string  // Respuesta a la pregunta del usuario
	"abreviatura": string  // ¿Cúal es la abreviatura de la moneda?
}
```


In [43]:
prompt = PromptTemplate(
    template= "Contesta la pregunta del usaurio de la mejor manera posible.\n{formato_instrucciones}\n{query}",
    input_variables= ["query"],
    partial_variables= {"formato_instrucciones": formato_instrucciones}
)

In [44]:
print(prompt)

input_variables=['query'] input_types={} partial_variables={'formato_instrucciones': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"moneda": string  // Respuesta a la pregunta del usuario\n\t"abreviatura": string  // ¿Cúal es la abreviatura de la moneda?\n}\n```'} template='Contesta la pregunta del usaurio de la mejor manera posible.\n{formato_instrucciones}\n{query}'


In [45]:
prompt = prompt.format(query= "¿Cuál es la moneda actual de Perú?")

In [46]:
output = llm.invoke(prompt)

In [47]:
print(output.content)

```json
{
	"moneda": "Nuevo Sol",
	"abreviatura": "PEN"
}
```
